In [50]:
from bs4 import BeautifulSoup
import urllib
import re
import json
import ast
import pandas as pd
import numpy as np


def get_data(soup):
    row_list = list()
    for container in soup.findAll("div", attrs={"data-id": re.compile("search-results-container")}):
        for doc in container.findAll("div", attrs={"class": re.compile("panel panel-default")}):
            row = dict()
            div = doc.findAll("div",attrs={"class": re.compile("panel-body rank-element padding-top-2 padding-bottom-1")},)[0]
            row["id"] = int(div.attrs["data-eecommerce-id"])
#             row["data_eecommerce_name"] = div.attrs["data-eecommerce-name"]
#             row["data_object_lat"] = float(div.attrs["data-object-lat"])
#             row["data_object_lon"] = float(div.attrs["data-object-lon"])
#             row["data_object_type"] = div.attrs["data-object-type"]
#             span = doc.findAll("span")[0]
#             row["link"] = span.attrs["data-href"]
            span = doc.findAll("span")[1]
            row["name"] = span.get_text().strip()
#             h4 = doc.findAll("h4")[0]
#             row["specialisations"] = str(h4.get_text().strip().replace("\t", "").replace("\n", "").split(","))
#             span = doc.findAll("span", attrs={"data-eecommerce-action": re.compile("product-click")})
#             if len(span)>2:
#                 row["opinion_counter"] = [np.int32(s) for s in span[2].get_text().replace("(", "").split() if s.isdigit()][0]
#             else:
#                 row["opinion_counter"] = None
            row_list.append(row)
    return row_list


def get_url(city, spec, main_url):
    url = main_url + spec + "/" + city + "/1"
    return url


def scrap_pages(url, max_pages=20):
    soups = []
    for _ in range(MAX_PAGES):
        try:
            html_page = urllib.request.urlopen(url)
        except urllib.error.HTTPError as e:
            print('HTTPError: {}'.format(e.code))
            break
        except urllib.error.URLError as e:
            print('URLError: {}'.format(e.reason))
            break
        else:
            page_no = url.split("/")[-1]
            print(page_no, end=" ")
            soup = BeautifulSoup(html_page)
            soups.append(soup)
            ul = soup.findAll("ul", attrs={"class": re.compile("pagination pagination-lg pagination-mobile offset-top-2 offset-bottom-2")},)
            if len(ul)>0:
                next_page_li = ul[0].findAll("li", attrs={"class": re.compile("next")})
            else:
                break
            if len(next_page_li):
                next_page_url = next_page_li[0].a["href"]
                url = next_page_url
            else:
                break
    print("\n")
    return soups

In [54]:
%%time
specs = pd.read_csv("specialisations.csv").values[:,0].tolist()
cities = pd.read_csv("cities.csv").values[:,0].tolist()
main_url = "https://www.znanylekarz.pl/"
row_list = []
for city in cities:
    print("city:", city)
    for spec in specs:
        print("\tspec:", spec, end='\n\t\tpages: ')
        url = get_url(city, spec, main_url)
        soups = scrap_pages(url, max_pages= 5)
        for soup in soups:
            row_list += get_data(soup)

city: warszawa
	spec: lekarz-w-trakcie-specjalizacji
		pages: 1 2 3 4 5 

	spec: lekarz-medycyny-pracy
		pages: 1 2 3 4 5 

	spec: lekarz-medycyny-estetycznej
		pages: 1 2 3 4 5 

	spec: lekarz-rehabilitacji-medycznej
		pages: 1 2 3 4 5 

	spec: specjalista-medycyny-naturalnej
		pages: 1 

	spec: lekarz-medycyny-ratunkowej
		pages: 1 

	spec: lekarz-chorob-zakaznych
		pages: 1 2 

	spec: chirurg-stomatologiczny
		pages: 1 2 3 

	spec: chirurg-szczekowy
		pages: 1 2 3 

	spec: endokrynolog-dzieciecy
		pages: 1 

	spec: okulista-dzieciecy
		pages: 1 2 

	spec: chirurg-naczyniowy
		pages: 1 2 3 4 5 

	spec: chirurg-plastyczny
		pages: 1 2 3 4 

	spec: kardiolog-dzieciecy
		pages: 1 2 

	spec: urolog-dzieciecy
		pages: 1 

	spec: ginekologia-dziecieca
		pages: 1 

	spec: audiolog
		pages: HTTPError: 404


	spec: foniatra
		pages: HTTPError: 404


	spec: biegly-sadowy
		pages: 1 2 

	spec: chirurg-dzieciecy
		pages: 1 2 3 4 5 

	spec: gastrolog-dzieciecy
		pages: 1 

	spec: higienistka-stom

IncompleteRead: IncompleteRead(128187 bytes read)

In [45]:
%%time
specs = pd.read_csv("specialisations.csv").values[:,0].tolist()
cities = pd.read_csv("cities.csv").values[:,0].tolist()
row_list = []
for city in cities:
    print("city:", city)
    for spec in specs:
        print("\tspec:", spec, end='\n\t\tpages: ')
        url = get_url(city, spec, "https://www.znanylekarz.pl/")
        soups = scrap_pages(url, 5)
        for soup in soups:
            row_list += get_data(soup)

city: warszawa
	spec: lekarz-w-trakcie-specjalizacji
		pages: 1,2,3,4,5

	spec: lekarz-medycyny-pracy
		pages: 1,2,3,4,5

	spec: lekarz-medycyny-estetycznej
		pages: 1,2,3,4,5

	spec: lekarz-rehabilitacji-medycznej
		pages: 1,2,3,4,5

	spec: specjalista-medycyny-naturalnej
		pages: 1

	spec: lekarz-medycyny-ratunkowej
		pages: 1

	spec: lekarz-chorob-zakaznych
		pages: 1,2

	spec: chirurg-stomatologiczny
		pages: 1,2,3

	spec: chirurg-szczekowy
		pages: 1,2,3

	spec: endokrynolog-dzieciecy
		pages: 1

	spec: okulista-dzieciecy
		pages: 1,2

	spec: chirurg-naczyniowy
		pages: 1,2,3,4,5

	spec: chirurg-plastyczny
		pages: 1,2,3,4

	spec: kardiolog-dzieciecy
		pages: 1,2

	spec: urolog-dzieciecy
		pages: 1

	spec: ginekologia-dziecieca
		pages: 1

	spec: audiolog
		pages: 

HTTPError: HTTP Error 404: Not Found

In [55]:
df = pd.DataFrame(row_list)
display(df.shape)
display(df)

(29912, 2)

,id,name
0,342226,lek. Marta Błażewicz
1,331630,lek. Paweł Marczak
2,301340,lek. Aleksandra Olejniczak
3,290786,lek. dent. Michał Kaczmarek
4,321682,lek. Dana Pavlovschi
...,...,...
29907,207522,Beata Elżbieta Świerczyńska
29908,235394,Janina Zielińska-Damska
29909,215490,lek. Jarosław Wojciech Czerniak
29910,245548,Agnieszka Ewa Jarzemska


In [57]:
import pandas_profiling
report = df.profile_report()
report.to_file(output_file="report.html")
report

In [37]:
df = df.drop_duplicates(subset='name')
print(df.shape[0])
print(df['id'].unique().shape[0])
print(df['name'].unique().shape[0])

22534
22534
22534
